In [1]:
from pandas import MultiIndex, DataFrame, Series, IndexSlice, Index
import numpy as np
import time
ix = IndexSlice

# Pandas Indexing with `.loc` and `.reindex`

Using **indexes** to subselect data via rows or columns is a very powerful feature of pandas and it is worth learning what an **index** is and how it can be utilised for optimal effect.

`.loc` is a flexible indexer for DataFrames and Series, operable either on the index or columns (in a DataFrame) or both simultaneously, although the simultaneous case necessarily requires **only a basic intersection (separable with a comma)**, so, without loss of generality, we can describe many of the features as they apply to an index of a Series. See below for some very basic, simple examples.

#### Series

In [2]:
ser = Series([1,2,3], index=["a", "b", "c"], name="example")
ser

a    1
b    2
c    3
Name: example, dtype: int64

In [3]:
ser.loc["a"]

1

In [4]:
ser.loc[["a", "b"]]

a    1
b    2
Name: example, dtype: int64

#### DataFrames

In [5]:
df = DataFrame([[1,2,3], [4,5,6], [7,8,9]], index=["a", "b", "c"], columns=["A", "B", "C"])
df

,A,B,C
a,1,2,3
b,4,5,6
c,7,8,9


In [6]:
df.loc["a"]

A    1
B    2
C    3
Name: a, dtype: int64

In [7]:
df.loc["a", "A"]

1

In [8]:
df.loc[["a", "b"], ["A", "B"]]

,A,B
a,1,2
b,4,5


## What is an Index?

Let's get to grips with the core structure of an **index**.

An `Index` is a **sequence** of defined `keys` (alternatively called `labels`).
There are **four** categories of index based on the properties it might have.

In [9]:
# HIDDEN notebook cell
idx = Index(["Unique", "Duplicated (Non-Unique)"], name="Key Values")
col = Index(["Monotonic", "Unsorted (Non-Monotonic)"], name="Key Sorting")
df = DataFrame([["All features available", "Certain features unavailable"], ["Certain features unavailable", "Least number supported features"]], index=idx, columns=col)
table = df.style

In [10]:
table

Key Sorting,Monotonic,Unsorted (Non-Monotonic)
Key Values,,
Unique,All features available,Certain features unavailable
Duplicated (Non-Unique),Certain features unavailable,Least number supported features


 - ``Monotonic`` means all the keys are sorted, whilst unsorted means they can be in any order.
 - ``Duplicated`` means at least one key is repeated in the index.
 
The following will return a ``KeyError`` since the key location is ambiguous for the duplicated key.

In [11]:
ser_unsort = Series([1,2,3,4], index=["a", "c", "b", "c"])
# ser_un.loc[ix["a":"c"]]  -> KeyError: "Cannot get right slice bound for non-unique label: 'c'"

However, an index can be sorted and then a slicer will work.

In [12]:
ser_mono = ser_unsort.sort_index()
ser_mono.loc[ix["a":"c"]]

a    1
b    3
c    2
c    4
dtype: int64

An index is different to a generic column or row since much functionality is constructed around being able to subselect data efficiently, and this depends on those properties of an index.

For an `Index` (and, specifically, not a `MultiIndex`) a key will either be contained in the index or not, and this often determines the error reporting upon indexing.

## What is a Key (or Label)?

Keys of an ``Index`` can be hashable (immutable) objects such as strings, integers, floats, complex, or tuples, but it is always **recommended** to maintain the same dtype or structure to avoid any indexing or sorting issues. Keys cannot be lists, sets or dicts.  

In [13]:
# Series([1,2,3], index=[["a", 1], {"a": 1}, {"a", 1}])  # TypeError: unhashable types: 'list', 'dict', 'set'
Series([1,2,3,4,5], index=["A", 2, 1.2, 1+1j, ("a", 1)])

A         1
2         2
1.2       3
(1+1j)    4
(a, 1)    5
dtype: int64

## Subselecting Keys in an Index Using `.loc`

There are different input formats to use in `.loc` which will result in different output formats.

 - A single explicit `key`, which may be unique or duplicated: `ser.loc["a"]`
 - A list of explicit `keys`, again these may be unique or duplicated: `ser.loc[["a", "b"]]`
 - A Python slice (which may fail for **unsorted and duplicated** indexes): `ser.loc[slice("a", "b")]`, or equivalently an `IndexSlice` which provides some syntactic sugar: `ser.loc[IndexSlice["a":"b"]]`
 - A colon, ':', which is syntactic sugar for a none slice along a whole axis and returns all values: `ser.loc[:]`
 - A boolean array-like of the same length of the index determining presence in the output: `ser.loc[[True, True, False]]`
 
Depending which input format is used the output format may be different, for example a `Series` might collapse to a `Scalar` or a `DataFrame` might collapse to a `Series` or even a `Scalar`. Whether or not `keys` are present in the `Index` may also determine whether a `KeyError` is generated.

Below we give examples of output formats with different input formats across different index properties.

In [14]:
# HIDDEN notebook cell

ret = None
def do(command):
    try:
        exec(f'global ret; ret={command}', globals())
    except KeyError:
        return 'KeyError'
    except ValueError:
        return 'ValueError'
    except IndexError:
        return 'IndexError'
    else:
        if isinstance(ret, (np.int64)):
            return 'int64'
        elif isinstance(ret, (Series)):
            if len(ret.index) == 0:
                return 'Empty Series'
            return 'Series'
        elif isinstance(ret, (DataFrame)):
            return 'DataFrame'
        return 'OtherType'

cases = [
("'a'", "single valid, unique key"),
("'!'", "single invalid key"), 
("['a']", "single valid, unique key as a list of valid keys"),
("['!']", "single invalid key as a list of invalid keys"),
("['a','c']", "list of valid, unique keys"),
("['a','!']", "list containing at least one invalid keys"),  
("ix['a':'c']", "Slice with valid, unique keys"),
("ix['a':'!']", "Slice with at least one invalid slice key"),
("ix['!':'!!']", "Slice with two invalid slice keys"),
("'b'", "Single valid, duplicated key (in duplicated Index)"),
("['b']", "Single valid, duplicated key as a list"),
("ix['b':'c']", "Slice with valid, duplicated keys"),
("ix['c':'b']", "Slice with valid, duplicated keys reversed"),
("[True, True, False, False]", "Boolean indexer of right length"),
("[True, False]", "Boolean indexer of wrong length")
]

commands = [f's.loc[{case[0]}]' for case in cases]

indexes = [
    Index(['a','c','b','d'], name='Unique Unsorted<br>["a","c","b","d"]'),
    Index(['a','c','b','d'], name='Unique Monotonic<br>["a","b","c","d"]').sort_values(),
    Index(['a','b','c','b'], name='Duplicated Unsorted<br>["a","b","c","b"]'),
    Index(['a','c','b','b'], name='Duplicated Monotonic<br>["a","b","b","d"]').sort_values(),
]

results = DataFrame('', 
                       index=Index([case[0] for case in cases], name="series.loc[{}]"), 
                       columns=[index.name for index in indexes]+["description"])
results.columns.name = "Index Properties"
results.loc[:, "description"] = [case[1] for case in cases]
for j, index in enumerate(indexes):
    s = Series([1,2,3,4], index=index)
    for i, command in enumerate(commands):
        results.iloc[i, j] = do(command)
    

def color(v):
    if v == "int64":
        return "color: #33C8FF"
    elif "Error" in v:
        return "color: #FF3333"
    elif v == "Series":
        return "color: #00C010"
    elif v == "Empty Series":
        return "color: #BADF03"
    return None
    
table = results.style.applymap(color, subset=results.columns[:4])

In [15]:
table

Index Properties,"Unique Unsorted[""a"",""c"",""b"",""d""]","Unique Monotonic[""a"",""b"",""c"",""d""]","Duplicated Unsorted[""a"",""b"",""c"",""b""]","Duplicated Monotonic[""a"",""b"",""b"",""d""]",description
series.loc[{}],,,,,
'a',int64,int64,int64,int64,"single valid, unique key"
'!',KeyError,KeyError,KeyError,KeyError,single invalid key
['a'],Series,Series,Series,Series,"single valid, unique key as a list of valid keys"
['!'],KeyError,KeyError,KeyError,KeyError,single invalid key as a list of invalid keys
"['a','c']",Series,Series,Series,Series,"list of valid, unique keys"
"['a','!']",KeyError,KeyError,KeyError,KeyError,list containing at least one invalid keys
ix['a':'c'],Series,Series,Series,Series,"Slice with valid, unique keys"
ix['a':'!'],KeyError,Empty Series,KeyError,Empty Series,Slice with at least one invalid slice key
ix['!':'!!'],KeyError,Empty Series,KeyError,Empty Series,Slice with two invalid slice keys


The example above shows that when there are multiple values to be returned, or a pattern that usually returns multiple values (such as a list of a single key: `['a']`), the result is a `Series`. However, the `Series` can collapse to a `Scalar` when a single `key` is given.

`KeyError` results when a `key` that does not exist in the `Index` is provided either as a single value or part of a list.

### Avoiding KeyErrors

Knowledge of which keys constitute an index may not always be known in some dynamic instances, particulary if aggregating data from a variety of different sources.

We can compare the `.loc` return to an alternative indexing function, `.reindex`, which is a method which only accepts a list of `keys`. There are no `KeyErrors` returned in this case, but the `.reindex` function cannot be used on duplicated indexes, so results in a `ValueError` instead for all cases.

For an index, reindexing also results in `NaN` values being added to the output for non-existent keys.

In [16]:
ser.reindex(["a", "b", "c", "!"])

a    1.0
b    2.0
c    3.0
!    NaN
Name: example, dtype: float64

In [17]:
# HIDDEN notebook cell

cases = [
("['a']", "single valid, unique key as a list of valid keys"),
("['!']", "single invalid key as a list of invalid keys"),
("['a','c']", "list of valid, unique keys"),
("['a','!']", "list containing at least one invalid keys"),  
("['b']", "Single valid, duplicated key as a list"),
]

commands = [f's.reindex({case[0]})' for case in cases]

results = DataFrame('', 
                       index=Index([case[0] for case in cases], name="series.reindex({})"), 
                       columns=[index.name for index in indexes]+["description"])
results.columns.name = "Index Properties"
results.loc[:, "description"] = [case[1] for case in cases]
for j, index in enumerate(indexes):
    s = Series([1,2,3,4], index=index)
    for i, command in enumerate(commands):
        results.iloc[i, j] = do(command)
    
table = results.style.applymap(color, subset=results.columns[:4])

In [18]:
table

Index Properties,"Unique Unsorted[""a"",""c"",""b"",""d""]","Unique Monotonic[""a"",""b"",""c"",""d""]","Duplicated Unsorted[""a"",""b"",""c"",""b""]","Duplicated Monotonic[""a"",""b"",""b"",""d""]",description
series.reindex({}),,,,,
['a'],Series,Series,ValueError,ValueError,"single valid, unique key as a list of valid keys"
['!'],Series,Series,ValueError,ValueError,single invalid key as a list of invalid keys
"['a','c']",Series,Series,ValueError,ValueError,"list of valid, unique keys"
"['a','!']",Series,Series,ValueError,ValueError,list containing at least one invalid keys
['b'],Series,Series,ValueError,ValueError,"Single valid, duplicated key as a list"


Therefore, if we have a `duplicated index`, the only way to avoid errors is to dynamically detect whether a list of `keys` are , or are not, contained within the index and use a boolean indexer. This will not add missing keys with `NaN` values.

In [19]:
ser = Series([1,2,3,4], index=["a", "b", "c", "b"])  # duplicated and unsorted
index_keys = ["a", "b", "!"]
ser[ser.index.isin(index_keys)]

a    1
b    2
b    4
dtype: int64

### Performance and Usage Of Various Indexing Strategies

Now we aim to test the different constructions for performance for the purposes of indexing large data. 

In [20]:
# HIDDEN notebook cell

cases = [
("s.loc[502]", "single valid, unique key"), 
("s.loc[[502]]", "single valid, unique key as a list of valid keys"),
("s.loc[[501,502]]", "list of valid keys"),  
("s.loc[ix[501:502]]", "Slice with valid keys"),
("s.loc[501]", "Single valid, duplicated key (in duplicated Index)"),
("s.loc[[501]]", "Single valid, duplicated key as a list"),
("s.loc[bool_indexer]", "Boolean indexer of right length"),
("s.reindex([502])", "single valid, unique key as a list of valid keys"),
("s.reindex([501,502])", "list of valid keys"), 
("s.reindex([501,502,999])", "list of with at least one invalid key"),
("s.loc[s.index.isin([502])]", "single valid, unique key as a list of valid keys"),
("s.loc[s.index.isin([501,502])]", "list of valid keys"), 
("s.loc[s.index.isin([501,502,999])]", "list of with at least one invalid key"),   
]

commands = [f'{case[0]}' for case in cases]

vals = np.arange(500)
arrays = [
    np.random.choice(vals, size=500, replace=False),
    np.random.choice(vals, size=500, replace=False), 
    np.random.choice(vals, size=500, replace=True),
    np.random.choice(vals, size=500, replace=True), 
]
for array in arrays:
    array[498] = 501
    array[499] = 502
arrays[2][497] = 501
arrays[3][497] = 501
indexes = [
    Index(arrays[0], name='Unique Unsorted'),
    Index(arrays[1], name='Unique Monotonic').sort_values(),
    Index(arrays[2], name='Duplicated Unsorted'),
    Index(arrays[3], name='Duplicated Monotonic').sort_values(),
]


bool_indexer = np.random.choice([True, False], size=500)
results = DataFrame('', 
                       index=Index([case[0] for case in cases], name="command"), 
                       columns=[index.name for index in indexes]+["description"])
results.loc[:, "description"] = [case[1] for case in cases]
for j, index in enumerate(indexes):
    s = Series(np.arange(500), index=index)
    for i, command in enumerate(commands):
        z = 9e9
        for k in range(10):
            t = time.perf_counter()
            val = do(command)
            t = time.perf_counter() - t
            z = t if t < z else z
        if val == "KeyError" or val == "ValueError" or val == "IndexError":
            results.iloc[i, j] = val
        else:
            results.iloc[i, j] = z * 1000000
    
table = results.style.format(lambda x: '{:.0f} us'.format(x) if not isinstance(x, str) else x)

In [21]:
table

,Unique Unsorted,Unique Monotonic,Duplicated Unsorted,Duplicated Monotonic,description
command,,,,,
s.loc[502],32 us,30 us,49 us,38 us,"single valid, unique key"
s.loc[[502]],284 us,288 us,245 us,221 us,"single valid, unique key as a list of valid keys"
"s.loc[[501,502]]",287 us,287 us,248 us,222 us,list of valid keys
s.loc[ix[501:502]],65 us,124 us,117 us,75 us,Slice with valid keys
s.loc[501],33 us,33 us,126 us,83 us,"Single valid, duplicated key (in duplicated Index)"
s.loc[[501]],319 us,302 us,256 us,216 us,"Single valid, duplicated key as a list"
s.loc[bool_indexer],125 us,102 us,96 us,119 us,Boolean indexer of right length
s.reindex([502]),280 us,265 us,ValueError,ValueError,"single valid, unique key as a list of valid keys"
"s.reindex([501,502])",325 us,265 us,ValueError,ValueError,list of valid keys


So the performance of either indexing option is not vastly different when to use either depends on the index properties and the intended result of the user.

In [22]:
table = DataFrame([
    ["<ul><li>Flexible input, adaptable output</li><li>Well documented</li></ul>", 
     "<ul><li>Yields a complete index, with explicit missing data for missing keys</li></ul>",
     "<ul><li>Avoids KeyError and does not return missing keys</li></ul>"],
    ["<ul><li>KeyErrors for dynamic missing keys</li></ul>",
     "<ul><li>Rigid input only allows list of keys</li><li>Rigid output</li></ul>",
     "<ul><li>Rigid input only allows list of keys</li><li>Clumsy syntax</li><li>Rigid output</li></ul>"]
], columns=[".loc", ".reindex", ".isin boolean"], index=["Advantages", "Disadvantages"])
table.style

,.loc,.reindex,.isin boolean
Advantages,"Flexible input, adaptable outputWell documented","Yields a complete index, with explicit missing data for missing keys",Avoids KeyError and does not return missing keys
Disadvantages,KeyErrors for dynamic missing keys,Rigid input only allows list of keysRigid output,Rigid input only allows list of keysClumsy syntaxRigid output


## MultiIndexing

Now we will discuss a `MultiIndex`, as opposed to a standard `Index`. With hierarchical, or multiindexing, comes a lot greater functionality, but at the expense of added complexity. For big data the use of multiindexes greatly improves the performance of subselecting data.

### What is a MultiIndex?

A `MultiIndex` is the same as an `Index` in that it is a **sequence** of `keys`. Again those keys may be unique or duplicated and monotonic or unsorted, just like the properties of a single index.

The difference is that each key has multiple `level_values` that comprise it. For example `("Mark", "September")` is a key in a multiindex whose two levels are made up of (first names, birth month).

In [23]:
midx = MultiIndex.from_tuples([("Jane", "June"), ("Mark", "September"), ("Jane", "April")], names=["name", "birth"])
midx

MultiIndex([('Jane',      'June'),
            ('Mark', 'September'),
            ('Jane',     'April')],
           names=['name', 'birth'])

This multiindex is currently unique, since no key is repeated but it is unsorted. We can fix that now if necessary.

In [24]:
midx.sort_values()

MultiIndex([('Jane',     'April'),
            ('Jane',      'June'),
            ('Mark', 'September')],
           names=['name', 'birth'])

In pandas there are 3 ways to create `MultIndex` objects.

#### MultiIndex.from_tuples

This is the way we have used above. An explicit list of tuples as keys, each with exactly the same number of elements corresponding to the number of levels, is provided. 

#### MultiIndex.from_array

This uses multiple arrays of the same length, each corresponding to a level, and zips corresponding elements together. This is the type of aggregation used if one were to create an index from multiple columns in a DataFrame, for example.

In [25]:
MultiIndex.from_arrays([["Mark", "Jane", "Jane"], ["September", "June", "April"]], names=["name", "birth"])

MultiIndex([('Mark', 'September'),
            ('Jane',      'June'),
            ('Jane',     'April')],
           names=['name', 'birth'])

#### MultiIndex.from_product

This uses multiple arrays and performs a Cartesian product, $Array1 \times Array2$, to reveal all combinations. In some circumstances this is warranted, but for large numbers of levels and elements this may produce too many combinations to be computationally feasible. Even in our small example it results in 3 unuseful entries.

In [26]:
MultiIndex.from_product([["Mark", "Jane"], ["September", "June", "April"]], names=["name", "birth"])

MultiIndex([('Mark', 'September'),
            ('Mark',      'June'),
            ('Mark',     'April'),
            ('Jane', 'September'),
            ('Jane',      'June'),
            ('Jane',     'April')],
           names=['name', 'birth'])

Once we have our MultiIndex we can interrogate its individual levels. For example we can see which unique elements are present in each level, and we can also get the associated ordering of those elements measured over the MultiIndex (where they may be duplicated, e.g. "Jane").

In [27]:
print("Level 0 Elements: ", midx.levels[0])
print("Level 1 Elements: ", midx.levels[1])
print("Level 0 Ordered Values: ", midx.get_level_values(0))
print("Level 1 Ordered Values: ", midx.get_level_values(1))

Level 0 Elements:  Index(['Jane', 'Mark'], dtype='object', name='name')
Level 1 Elements:  Index(['April', 'June', 'September'], dtype='object', name='birth')
Level 0 Ordered Values:  Index(['Jane', 'Mark', 'Jane'], dtype='object', name='name')
Level 1 Ordered Values:  Index(['June', 'September', 'April'], dtype='object', name='birth')


We can also `reindex` a level of a MultiIndex but this does not add any new keys to the MultiIndex. For example:

In [28]:
midx = midx.reindex(["Jane", "Mark", "Michael"], level=0)[0]
midx

MultiIndex([('Jane',      'June'),
            ('Jane',     'April'),
            ('Mark', 'September')],
           names=['name', 'birth'])

But if we interrogate the unique values present for level0 then "Michael" has been added as a new component. It has essentially become an `unused_level_value` where it is present as a value within the level elements but is **not present** in any key combination.

In [29]:
print("Level 0 New Elements: ", midx.levels[0])

Level 0 New Elements:  Index(['Jane', 'Mark', 'Michael'], dtype='object', name='name')


### Summarising Terminolgy for a MultiIndex

 - `MultiIndex` is a sequence of `Keys`.
 - `Key` is a tuple containing valid hashable types for each `Level` of the tuple, e.g. ("Jane", "April").
 - `Level` refers to a subcomponent of a `Key` based on the 1st, 2nd tuple index, etc.
 - `Level Elements` are the unique values of a specific `Level` which may (or may not) be part of any single `Key` in the `MultiIndex`. If it is part of a `Key` it is a `Used Level Element` otherwise it is an `Unused Level Element`.
 - `Level Values`  are the sequence of `Used Level Elements` that form the part of the sequence of `Keys` in the `MultiIndex`
 
 Why is it important to define any of these terms? Becuase to know how to construct, reindex and select the data efficiently and dynamically it is useful to know the composition structure of MultiIndexes.
 
Since MultiIndex keys are composed of level elements this gives rise to combinations which may or may not be a key in the MultiIndex.

In [30]:
result = DataFrame([
    ["Maybe a Key", "Never be a Key", "Erroneous Key"],
    ["Never be a Key", "Never be a Key", "Errorneous Key"],
    ["Erroneous Key", "Erroneous Key", "Erroneous Key"]
], columns=["Used", "Unused", "Erroneous"], index=["Used", "Unused", "Erroneous"])
result.index.name = "Level 0 Element"
result.columns.name = "Level 1 Element"
table = result.style

In [31]:
table

Level 1 Element,Used,Unused,Erroneous
Level 0 Element,,,
Used,Maybe a Key,Never be a Key,Erroneous Key
Unused,Never be a Key,Never be a Key,Errorneous Key
Erroneous,Erroneous Key,Erroneous Key,Erroneous Key


Even when both level0 and level1 elements are used, unless the index is a full Cartesian product of the level elements (which is a rare case) there will most likely be cases where two used level elements combine to result in a key that is not present in the index, for example `("Jane", "September)` or `("Mark", "April)`. Whereas we can know for sure that `("Sheena","April")` is not a key since `"Sheena"` is an erroneous level0 element, and furthermore, even though `"Michael"` may have been added to the level0 elements it is still usused so we are also sure it will not produce a valid key.

## Subselecting Keys in an Index Using .loc

Similar to an `Index`. There are different input formats to use in .loc which will result in different output formats.

 - A single explicit key, which may be unique or duplicated: ser.loc[("Jane", "June")]
 - A list of explicit keys, again these may be unique or duplicated: ser.loc[[("Jane", "June"), ("Jane", "April")]]
 - A Python slice (which may fail for unsorted and/or duplicated indexes): ser.loc[slice(("Jane", "April"), ("Jane",    "June")], or equivalently an IndexSlice which provides some syntactic sugar: ser.loc[IndexSlice[("Jane", "April"):("Jane", "June")]]
 - A colon, ':', which is syntactic sugar for a none slice along a whole axis and returns all values: ser.loc[:]
 - A boolean array-like of the same length of the index determining presence in the output: ser.loc[[True, True, False]]

When indexing a MultiIndex there is also the ability to subselect via specific levels:

 - By providing an explicit level element: ser.loc[(slice(None), "April")]
 - By providing a list of level elements: ser.loc[(slice(None), ["April", "September"])]
 - By providing either a python or IndexSlice: ser.loc[(slice(None), IndexSlice["April":"September"])]
 - In combination with a boolean indexer: ser.loc[("Jane", [True, False, True])]

In our example above the leading levels where no indexing is performed must be identified by `slice(None)`. However, for a MultiIndex with many levels, any missing levels will be inferred to be `slice(None)` if not explicitly identified. For example `ser.loc["Jane"]` is equivalent to `ser.loc[("Jane", slice(None))]`.

In [32]:
indexes = [
    MultiIndex.from_tuples(names=['Unique Levels Unsorted', None],
        tuples=[('a', 0), ('c', 2), ('b', 1), ('d', 3)], ),
    MultiIndex.from_tuples(names=['Unique Index Unsorted', None],
        tuples=[('a', 0), ('b', 1), ('c', 2), ('b', 3)], ),
    MultiIndex.from_tuples(names=['Unique Levels Monotonic', None],
        tuples=[('a', 0), ('c', 2), ('b', 1), ('d', 3)], ).sort_values(),
    MultiIndex.from_tuples(names=['Unique Index Monotonic', None],
        tuples=[('a', 0), ('b', 1), ('c', 2), ('b', 3)], ).sort_values(),
    MultiIndex.from_tuples(names=['Duplicated Unsorted', None],
        tuples=[('a', 0), ('b', 1), ('c', 2), ('b', 1)], ),
    MultiIndex.from_tuples(names=['Duplicated Monotonic', None],
        tuples=[('a', 0), ('b', 1), ('c', 2), ('b', 1)], ).sort_values(),
]


cases = [
("('a',0)", "single valid, unique key"),
("('!',0)", "single invalid key"), 
("[('a',0)]", "single valid, unique key as a list of valid keys"),
("[('!',0)]", "single invalid key as a list of invalid keys"),
("[('a',0),('c',2)]", "list of valid, unique keys"),
("[('a',0),('!',0)]", "list containing at least one invalid keys"),  
("ix[('a',0):('c',2)]", "Slice with valid, unique keys"),
("ix[('a',0):('!',0)]", "Slice with at least one invalid slice key"),
("ix[('!',0):('!',2)]", "Slice with two invalid slice keys"),
("('b',1)", "Single valid, duplicated key (in duplicated Index)"),
("[('b',1)]", "Single valid, duplicated key as a list"),
("ix[('b',1):('c',2)]", "Slice with valid, duplicated keys"),
("ix[('c',2):('b',1)]", "Slice with valid, duplicated keys reversed"),
("[True, True, False, False]", "Boolean indexer of right length"),
("[True, False]", "Boolean indexer of wrong length")
]

commands = [f's.loc[{case[0]}]' for case in cases]

results = DataFrame('', 
                       index=Index([case[0] for case in cases], name="series.loc[{}]"), 
                       columns=[index.names[0] for index in indexes]+["description"])
results.columns.name = "Index Properties"
results.loc[:, "description"] = [case[1] for case in cases]
for j, index in enumerate(indexes):
    s = Series([1,2,3,4], index=index)
    for i, command in enumerate(commands):
        results.iloc[i, j] = do(command)
        
table = results.style.applymap(color, subset=results.columns[:6])

<ipython-input-14-d323a95a38ca>:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(f'global ret; ret={command}', globals())


In [33]:
table

Index Properties,Unique Levels Unsorted,Unique Index Unsorted,Unique Levels Monotonic,Unique Index Monotonic,Duplicated Unsorted,Duplicated Monotonic,description
series.loc[{}],,,,,,,
"('a',0)",int64,int64,int64,int64,Series,Series,"single valid, unique key"
"('!',0)",KeyError,KeyError,KeyError,KeyError,KeyError,KeyError,single invalid key
"[('a',0)]",Series,Series,Series,Series,Series,Series,"single valid, unique key as a list of valid keys"
"[('!',0)]",KeyError,KeyError,KeyError,KeyError,KeyError,KeyError,single invalid key as a list of invalid keys
"[('a',0),('c',2)]",Series,Series,Series,Series,Series,Series,"list of valid, unique keys"
"[('a',0),('!',0)]",KeyError,KeyError,KeyError,KeyError,KeyError,KeyError,list containing at least one invalid keys
"ix[('a',0):('c',2)]",KeyError,KeyError,Series,Series,KeyError,Series,"Slice with valid, unique keys"
"ix[('a',0):('!',0)]",KeyError,KeyError,Empty Series,Empty Series,KeyError,Empty Series,Slice with at least one invalid slice key
"ix[('!',0):('!',2)]",KeyError,KeyError,Empty Series,Empty Series,KeyError,Empty Series,Slice with two invalid slice keys


In [34]:
# HIDDEN cell

A = ["d", "b", "c", "a"]
B = ["a", "b", "c", "b"]

# zip(A,A) = unique index with unique level elements
# zip(A,B) = unique index with duplicated level elements
# zip(B,B) = duplicated index with duplicated level elements

indexes = [
    MultiIndex.from_arrays(names=['Unique Levels Unsorted', None],
        arrays=[A, A]),
    MultiIndex.from_arrays(names=['Unique Index Unsorted', None],
        arrays=[A, B]),
    MultiIndex.from_arrays(names=['Unique Levels Monotonic', None],
        arrays=[A, A]).sort_values(),
    MultiIndex.from_arrays(names=['Unique Index Monotonic', None],
        arrays=[A, B]).sort_values(),
    MultiIndex.from_arrays(names=['Duplicated Unsorted', None],
        arrays=[B, B]),
    MultiIndex.from_arrays(names=['Duplicated Monotonic', None],
        arrays=[B, B]).sort_values(),
]


cases = [
# ("slice(None)", "None Slice", "A"),
# ("'a'", "single valid, unique key", "B"),
# ("'!'", "single invalid key", "C"), # Always gives KeyError
("['a']", "single valid, unique key as a list of valid keys", "D"),
("['!']", "single invalid key as a list of invalid keys", "E"),
# ("['a','c']", "list of valid, unique keys", "F"),
("['a','!']", "list containing at least one invalid keys", "G"),  
# ("ix['a':'c']", "Slice with valid, unique keys", "H"),
# ("ix['a':'!']", "Slice with at least one invalid slice key", "I"),
# ("ix['!':'!!']", "Slice with two invalid slice keys", "J"),
# ("'b'", "Single valid, duplicated key (in duplicated Index)", "K"),
# ("['b']", "Single valid, duplicated key as a list", "L"),
# ("ix['b':'c']", "Slice with valid, duplicated keys", "M"),
# ("ix['c':'b']", "Slice with valid, duplicated keys reversed", "N"),
("[True, True, False, False]", "Boolean indexer of right length", "O"),
("[True, False, True, False]", "Boolean indexer of right length (diff)", "P"),
# ("[True, False]", "Boolean indexer of wrong length", "Q")  # Always gives ValueError
]

cases2 = [
("slice(None)", "None Slice", "A"),
# ("'a'", "single valid, unique key", "B"),
# # ("'!'", "single invalid key", "C"), # Always gives KeyError
# ("['a']", "single valid, unique key as a list of valid keys", "D"),
# ("['!']", "single invalid key as a list of invalid keys", "E"),
# ("['a','c']", "list of valid, unique keys", "F"),
("['a','!']", "list containing at least one invalid keys", "G"),  
# ("ix['a':'c']", "Slice with valid, unique keys", "H"),
# ("ix['a':'!']", "Slice with at least one invalid slice key", "I"),
# ("ix['!':'!!']", "Slice with two invalid slice keys", "J"),
# ("'b'", "Single valid, duplicated key (in duplicated Index)", "K"),
# ("['b']", "Single valid, duplicated key as a list", "L"),
# ("ix['b':'c']", "Slice with valid, duplicated keys", "M"),
# ("ix['c':'b']", "Slice with valid, duplicated keys reversed", "N"),
# ("[True, True, False, False]", "Boolean indexer of right length", "O"),
# ("[True, False, True, False]", "Boolean indexer of right length (diff)", "P"),
# ("[True, False]", "Boolean indexer of wrong length", "Q")  # Always gives ValueError
]

commands = []
labels = []
codes = []
for l0_case in cases:
    for l1_case in cases2:
        labels.append(f"({l0_case[0]}, {l1_case[0]})")
        codes.append(f"{l0_case[2]}{l1_case[2]}")
        commands.append(f's.loc[({l0_case[0]}, {l1_case[0]})]')
        
results = DataFrame('', 
                       index=Index(labels, name="series.loc[{}]"), 
                       columns=[index.names[0] for index in indexes])
results.columns.name = "Index Properties"
# results.loc[:, "description"] = [case[1] for case in cases]
for j, index in enumerate(indexes):
    s = Series([1,2,3,4], index=index)
    for i, command in enumerate(commands):
        results.iloc[i, j] = do(command)
        
def codes_checker(s, match, error):
    match_bool = np.array([match in code for code in codes])
    error_bool = s == error
    return ["background-color: green" if x else "" for x in (match_bool & error_bool)]
    
        
table = results.style.applymap(color, subset=results.columns[:6])\
               .apply(codes_checker, match="C", error="KeyError") # single invalid should always give KeyError

In [35]:
table

Index Properties,Unique Levels Unsorted,Unique Index Unsorted,Unique Levels Monotonic,Unique Index Monotonic,Duplicated Unsorted,Duplicated Monotonic
series.loc[{}],,,,,,
"(['a'], slice(None))",Series,Series,Series,Series,Series,Series
"(['a'], ['a','!'])",Series,KeyError,Series,KeyError,Series,Series
"(['!'], slice(None))",Empty Series,Empty Series,Empty Series,Empty Series,Empty Series,Empty Series
"(['!'], ['a','!'])",Empty Series,Empty Series,Empty Series,Empty Series,Empty Series,Empty Series
"(['a','!'], slice(None))",Series,Series,Series,Series,Series,Series
"(['a','!'], ['a','!'])",Series,KeyError,Series,KeyError,Series,Series
"([True, True, False, False], slice(None))",Series,Series,Series,Series,Series,Series
"([True, True, False, False], ['a','!'])",KeyError,Series,Series,KeyError,Series,Series
"([True, False, True, False], slice(None))",Series,Series,Series,Series,Series,Series
